In [94]:
# Libraries
import pandas as pd
import datetime as dt
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [96]:
# Read in team name to 3 letter code dictionary
with open('../../data/team_name_dictionary.txt', 'r') as f:
    team_name_dict = json.load(f)

In [93]:

# Configure Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode

# Create a WebDriver instance
driver = webdriver.Chrome(options=options)

# URL to scrape
url = 'https://sportsbook.draftkings.com/leagues/hockey/nhl?category=game-lines&subcategory=60-min-line'

# Open the URL
driver.get(url)

# Explicitly wait for the elements with the specified class to appear
wait = WebDriverWait(driver, 15)  # Wait for up to 30 seconds

# Get each game card
game_cards = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'sportsbook-event-accordion__wrapper.expanded')))

# Only keep the game cards that are on today's date
month_mapping = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}

bet_options = []
odds = []

for card in game_cards:
    date = card.find_element(By.CLASS_NAME, 'sportsbook-event-accordion__date').text
    split_date = date.split()
    parts_to_keep = []
    for part in split_date:
        if part.endswith('AM') or part.endswith('PM') or part in ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']:
            continue
        elif part.endswith('TH'):
            part = part[:-2]
            parts_to_keep.append(part)
        else:
            parts_to_keep.append(month_mapping[part])
    date = dt.date(dt.datetime.now().year, int(parts_to_keep[0]), int(parts_to_keep[1]))
    if date == dt.date(2023, 10, 10):
        bet_options_single_game = card.find_elements(By.CLASS_NAME, 'sportsbook-outcome-cell__label')
        bet_options_single_game = [opt.text for opt in bet_options_single_game]
        bet_options.extend(bet_options_single_game)

        odds_single_game = card.find_elements(By.CLASS_NAME, 'sportsbook-odds.american.default-color')
        odds_single_game = [int(odd.text.replace("−", "-")) for odd in odds_single_game]
        odds.extend(odds_single_game)


print(bet_options)
print(odds)

# Quit the WebDriver
driver.quit()


['NSH Predators', 'Draw', 'TB Lightning', 'CHI Blackhawks', 'Draw', 'PIT Penguins', 'SEA Kraken', 'Draw', 'VGK Golden Knights']
[225, 340, -115, 275, 370, -140, 210, 330, -105]
